In [2]:
source("utils/plot.R")

In [3]:
data <- read_tsv(snakemake@input[["clues"]], col_types=cols(locus=col_integer())) %>%
    mutate(
        locus_str = paste0('Pos. ', scales::comma(locus))
    ) %>%
    filter(log_prob > quantile(log_prob, 0.75))

In [26]:
num_tree_colors <- 4

trees <- read_delim(snakemake@input[["arg_info"]], delim=';', col_types=cols()) %>%
    filter(is_not_mapping==0) %>%
    select(pos_of_snp, tree_index) %>%
    mutate(tree_colour=as.factor(tree_index %% num_tree_colors)) %>%
    rename(locus=pos_of_snp)

region_limits <- c(
    snakemake@config$sweep_region_start/1000,
    snakemake@config$sweep_region_end/1000
)

selstrens = data %>%
    group_by(locus) %>%
    slice_sample(n=1) %>%
    select(locus, s_estimate) %>%
    ungroup() %>%
    left_join(trees, by='locus')

transformed <- selstrens %>%
    filter(s_estimate > 0, s_estimate < max(s_estimate))

In [30]:
raw_fig <- ggplot(selstrens) +
    geom_point(aes(x=locus/1000, y=s_estimate)) +
    labs(x='Position (kb)', y='Estimated s') +
    scale_x_continuous(limits=region_limits, labels=scales::comma) +
    turkana_theme +
    turkana_colour +
    theme(
        legend.position='none'
    )
turkana_save(snakemake@output[["raw"]], raw_fig)

In [31]:
trans_fig <- ggplot(transformed) +
    geom_point(aes(x=locus/1000, y=s_estimate)) +
    labs(x='Position (kb)', y='Estimated s') +
    scale_x_continuous(limits=region_limits, labels=scales::comma) +
    turkana_theme +
    turkana_colour +
    theme(
        legend.position='none'
    )
turkana_save(snakemake@output[["transformed"]], trans_fig)

In [35]:
for (loc in unique(data$locus)) {
    
    this <- data %>% filter(locus==loc)
    title <- paste0('Pos. ', scales::comma(loc))
    subtitle <- paste0('Tree ', filter(trees, locus==loc)$tree_index[[1]])
    s <- this$s_estimate[[1]]
    
    clues <- ggplot(this) +
        stat_summary_hex(aes(x=gen, y=freq, z=exp(log_prob), alpha=exp(log_prob)), fun='sum', bins=12) +
        geom_label(
            aes(x=max(gen), y=1.0, label=paste0('s = ', s)),
            size=2.5,
            hjust=0,
            family='Arial Narrow'
        ) +
        scale_y_continuous(limits=c(0, 1)) +
        scale_x_reverse() +
        scale_fill_gradient(low='lightgrey', high='black') +
        labs(
            title=title,
            subtitle=subtitle,
            x='Generations ago',
            y='Allele freq.'
        ) +
        turkana_theme +
        theme(
            legend.position = 'none',
            axis.text.x=element_text(angle=45, hjust=1),
            plot.subtitle=element_text(size=10)
        )

    turkana_save(
        paste0(snakemake@params[["plot_dir"]], "/clues_", loc, ".pdf"),
        clues, asp=1, width=2
    )
    
}